## Steganography Voila Application
Note: Images and supporting text files for reading and writing need to be plaed in the './images/' folder. <br>
Open Source BSD 3-Clause License, 2020 and 2021, Author: William (Rick) Weitzel <br>
Please see license file and ReadMe file for further instructions.

In [1]:
# BSD 3-Clause License
# Copyright (c) 2020 and 2021
# Author: William (Rick) Weitzel

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer.

# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.

# 3. Neither the name of the copyright holder nor the names of its
#    contributors may be used to endorse or promote products derived from
#    this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.



In [2]:
print("Loading libraries and importing functions.") 
# load libraries
import steg_lib as sl
import matplotlib.pyplot as plt 
import numpy as np 
import os
import io
from PIL import Image
import ipywidgets as widgets
from ipywidgets import interact


Loading libraries and importing functions.


In [3]:
# define directory and type of image file to be used
file_directory = './images/'
substring = '.tiff'

In [4]:
# generate file list from file_directory (with exception handling)
file_list = []
try:
    all_files = os.listdir(file_directory)
except:
    print('something wrong with your directory')
for i in range(0,len(all_files)):
    if substring in all_files[i]:
        file_list.append(all_files[i])
if len(file_list) == 0:
    print('no files with this specification')
    print('place .tiff image file(s) in directory and try again')
file_list.sort() # needed to correct os.listdir() order
# file_list

In [5]:
output = widgets.Output()

button_select_file = widgets.Button(
    description="Click to confirm selected file",
    layout=widgets.Layout(width='50%',),
    icon='check')

file_select = widgets.Dropdown(
    options = file_list,
    value = file_list[0],
    description = 'Select File:',
    disabled = False,
    )

@output.capture(clear_output=False,wait=True)
def on_button1_clicked(b):
    global pic_array, im, pic_array
    im = Image.open(file_directory + file_select.value)
    pic_array = np.array(im)[:,:,:3] #drop the alpha channel, if present
    button_select_file.description = 'picture loaded'
    print("you selected: ", file_select.value)

button_select_file.on_click(on_button1_clicked)

display(file_select, button_select_file)
output

Dropdown(description='Select File:', options=('coded_image.tiff', 'sub_sun.tiff'), value='coded_image.tiff')

Button(description='Click to confirm selected file', icon='check', layout=Layout(width='50%'), style=ButtonSty…

Output()

In [6]:
output = widgets.Output()

input_message = widgets.Textarea(
    value='Starter message - edit as you like',
    placeholder='Type something',
    description='Message:',
    disabled=False,
    rows=1,
    layout=widgets.Layout(width='75%'))

button_check_message = widgets.Button(
    style = {'description_width': 'initial'},
    description='Check input message',
    tooltip='Click to display input message',
    layout=widgets.Layout(width='50%'),
    icon='check')

@output.capture(clear_output=True,wait=True)
def on_button2_clicked(_):
    button_check_message.description='Message checked'
    print(input_message.value)

button_check_message.on_click(on_button2_clicked)

display(button_check_message, input_message)
output


Button(description='Check input message', icon='check', layout=Layout(width='50%'), style=ButtonStyle(), toolt…

Textarea(value='Starter message - edit as you like', description='Message:', layout=Layout(width='75%'), place…

Output()

In [7]:
## convert message to binary

# output = widgets.Output()

# button_convert_message = widgets.Button(
#     style = {'description_width': 'initial'},
#     description='Convert message to binary',
#     tooltip='Click to convert message',
#     icon='check',
#     layout=widgets.Layout(width='75%'))

# @output.capture(clear_output=False,wait=True)
# def on_button3_clicked(b):
#     global binary_message, str_message_length
#     binary_message, str_message_length = \
#         sl.convert(input_message.value)
#     button_convert_message.description = 'message converted to binary'
#     print("message converted")

# button_convert_message.on_click(on_button3_clicked)

# display(button_convert_message)
# output


In [8]:
# insert binary segment in image

start_integer = widgets.IntText(
    value=42,
    description='Enter start location:',
    style = {'description_width': 'initial'},
    disabled=False
)

button_insert_message = widgets.Button(
    style = {'description_width': 'initial'},
    description='insert message in picture',
    tooltip='Click to insert message',
    icon='check',
    layout=widgets.Layout(width='75%'))

output = widgets.Output()

@output.capture(clear_output=False,wait=True)
def insert_message(image, message, start_loc):
    global str_message_length
    (binary_message, str_message_length) = sl.convert(message)
    (image_vector, carrier_segment, length_of_binary, 
        rows, cols, colors) = sl.prepare_carrier_segment(image, 
        binary_message, start_loc)
    coded_image_segment = sl.insert_message(carrier_segment, 
        binary_message, length_of_binary)
    coded_image_vector = sl.insert_coded_segment(image_vector,
        coded_image_segment,start_loc)
    coded_image = sl.get_coded_image(coded_image_vector,rows,cols,colors)
    # save new image with message, provide message length
    plt.imsave(file_directory + 'coded_image.tiff',coded_image, format = 'tiff')
    s = str(str_message_length)
    output_text = 'Message string length is: ' + s \
        + '\n and message start location is: ' + str(start_loc) \
        + '\n and the Message is : \n' + message 
    f = open(file_directory + 'message_info','w')
    f.write(output_text)
    f.close()
    
@output.capture(clear_output=True,wait=True)
def on_button4_clicked(_):
    start_int = int(start_integer.value)
    insert_message(pic_array, input_message.value, start_int)
    button_insert_message.description = 'binary message inserted'
    print("Message inserted, see 'coded_image.tiff' in " + file_directory)
    print('Message str length = ', str_message_length)
    print('Start location = ', start_int)
    print("and please see 'message_info' file in " + file_directory)

button_insert_message.on_click(on_button4_clicked)

display(start_integer, button_insert_message)
output


IntText(value=42, description='Enter start location:', style=DescriptionStyle(description_width='initial'))

Button(description='insert message in picture', icon='check', layout=Layout(width='75%'), style=ButtonStyle(),…

Output()

In [9]:
# decode the image
output = widgets.Output()

provide_start_integer = widgets.IntText(
    value=42,
    description='Enter start location:',
    style = {'description_width': 'initial'},
    disabled=False
)

provide_message_length = widgets.IntText(
    value=42,
    description='Enter message length:',
    style = {'description_width': 'initial'},
    disabled=False
)

button_read_message = widgets.Button(
    style = {'description_width': 'initial'},
    description='read message from picture',
    tooltip='Click to read message',
    icon='check',
    layout=widgets.Layout(width='75%'))

@output.capture(clear_output=True,wait=True)
def read_message(start_loc, message_length):
    global decoded_message, coded_image_segment, coded_image
    coded_image = plt.imread(file_directory + 'coded_image.tiff')
    coded_image = coded_image[:,:,:3]
    coded_image_segment = sl.get_coded_image_segment(coded_image,start_loc,message_length)
    binary_message = sl.extract_binary_message(coded_image_segment)
    decoded_message = sl.convert_back(binary_message)
    f = open(file_directory + 'decoded_message','w')
    f.write(decoded_message)
    f.close()
    
@output.capture(clear_output=True,wait=True)
def on_button5_clicked(_):
    start = int(provide_start_integer.value)
    length = int(provide_message_length.value)
    read_message(start, length)
    button_read_message.description = 'message decoded'
    print(decoded_message)
    print("also see 'decoded_message' file ")

button_read_message.on_click(on_button5_clicked)

display(provide_start_integer, provide_message_length, button_read_message)
output


IntText(value=42, description='Enter start location:', style=DescriptionStyle(description_width='initial'))

IntText(value=42, description='Enter message length:', style=DescriptionStyle(description_width='initial'))

Button(description='read message from picture', icon='check', layout=Layout(width='75%'), style=ButtonStyle(),…

Output()